MultiRAT preprocessing
================
Joanes Grandjean

![rat art](../assets/img/rat_art.png)

# Purpose
In this section I generate the bare bone BIDS folder metadata ahead of an upload to opennenuro.org. Datasets acquired with standardRAT protocol were generated using proper BIDS converter (bidscoin and brkraw), so they contain the right metadata. Other datasets were supplied `as is` by contributing parties. See   [asset preparation](scripts/proj_asset.ipynb) for the detail on how supplied data were converted to NIFTI and BIDS.   

If re-using some of the scripts, please follow citations guidelines for
the software used. I’ve provided the links to the software wherever
possible. See also the [license](../LICENSE.md) for this software.


In [3]:
# init variables
init_folder='/home/traaffneu/joagra/code/MultiRat'  # location of the codes
analysis_folder='/project/4180000.19/multiRat'      # location of the bids folder and results
df_path='../assets/table/meta_data_20210622.tsv'    # meta-data table


In [4]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(df_path, sep='\t')  # load the table

BIDs 1.6.0 requires the following for fMRI. 
1. RepetitionTime
2. TaskName (for func)

The rest is recommended or optional. I am however unsure of the precision of all parameters provided by the users for their respective datasets. For this reason, I only added the following meta-data items to the .json files. 

MagneticFieldStrength
PulseSequenceType
EchoTime

The full table with raw user inputs is provided in the ./assets/table folder. 

In [30]:
i = 1  # eventually make this as a loop. 


subject_ID="sub-0"+str(df.iloc[i]['rat.sub'])
session_ID="ses-"+str(df.iloc[i]['rat.ses'])

bidsversion='1.6.0'

magneticfield=df.iloc[i]['MRI.field.strength']

anatfilename=os.path.join(analysis_folder, 'bids_test',subject_ID,session_ID,'anat',subject_ID+'_'+session_ID+'_T2w.json')
anatTR=df.iloc[i]['anat.TR']
anatTE=df.iloc[i]['anat.TE']
anatseq=str(df.iloc[i]['anat.sequence'])

funcfilename=os.path.join(analysis_folder, 'bids_test',subject_ID,session_ID,'func',subject_ID+'_'+session_ID+'_task-rest_run-1_bold.json')
funcTR=df.iloc[i]['func.TR']
funcTE=df.iloc[i]['func.TE']
funcseq=str(df.iloc[i]['func.sequence'])

if df.iloc[i]['exp.type']=='resting-state':
    taskname='rest'
else:
    taskname='stim'
    taskonset=str(df.iloc[i]['func.sensory.onset'])
    taskduration=str(df.iloc[i]['func.sensory.duration'])

In [31]:
import json

func={
    'MagneticFieldStrength': magneticfield,
    'EchoTime': funcTE,
    'RepetitionTime': funcTR,
    'PulseSequenceType': funcseq
    }

anat={
    'MagneticFieldStrength': magneticfield,
    'EchoTime': anatTE,
    'RepetitionTime': anatTR,
    'PulseSequenceType': anatseq
    }

In [32]:

with open(anatfilename, 'w') as outfile:
    json.dump(anat, outfile)

with open(funcfilename, 'w') as outfile:
    json.dump(func, outfile)

In [39]:
# ! bids validator in python is moody. seemly working using website (minus some warnings)
from bids_validator import BIDSValidator
BIDSValidator().is_bids(os.path.join(analysis_folder,'bids_test'))


False